In [1]:
#libraries
import os
import pandas as pd
from datetime import datetime as dt
from datetime import time as t
from datetime import timedelta
from dateutil import relativedelta
import numpy as np
from IPython.display import display

In [2]:
# source
user_credential = r'/Users/dinhhoang.nguyen.CONCENTRIX/OneDrive - Concentrix Corporation\WFM-internal'

In [3]:
# import master roster
masterroster = pd.read_excel(os.path.join(r'C:', user_credential, r'Data\User\CNX Global Master Roster.xlsx'), sheet_name='Sheet1')

In [4]:
tl = pd.read_excel(os.path.join(r'C:', user_credential, r'Data\TL\TL List.xlsx'), sheet_name='Sheet1')

In [5]:
# Import extention
extension = pd.read_excel(os.path.join(r'C:', user_credential, r'Data\Extension\Extension.xlsx'), sheet_name='Sheet1')

In [6]:
# import the workplan
workplan_wst = pd.read_json(os.path.join(r'C:', user_credential, r'DB\filejson\workplan_wst.json'))
# add additional info of agents
workplan_wst = pd.merge(workplan_wst, tl[['Emp ID', 'Supervisor (no need fill)']], left_on = 'EID', right_on = 'Emp ID', how='left')

In [7]:
# timestamp = dt.now()
timestamp = dt(2023, 11, 20, 13, 27, 28)

In [8]:
# import the realtime view
rt_view = pd.read_excel(os.path.join(r'C:', user_credential, r'Data\Realtime view\00_RTA_View-Agent Team Real Time.xlsx'), sheet_name='00_RTA_View-Agent Team Real Tim')
rt_view = pd.merge(rt_view, extension[['Extension Number', 'Emp ID']], left_on='Extension', right_on='Extension Number', how='left')
rt_view = rt_view.drop(columns={'Extension Number'}) #to remove abundant col
rt_view = pd.merge(rt_view, masterroster[['CUIC Name', 'Employee_ID']], left_on='Agent', right_on='CUIC Name', how='left') #get agent's ID from master roster
rt_view['Emp ID'] = rt_view.apply(lambda x: x['Employee_ID'] if pd.isnull(x['Extension']) or pd.isnull(x['Emp ID']) else x['Emp ID'], axis=1) #replace ID from extension (which we ref from Extension) with ID from master roster (which we ref from CUIC name))
rt_view = rt_view.drop(columns={'Employee_ID'}) #remove this col since this no longer necessary
rt_view['time_stamp'] = timestamp # get current time stamp to compare with the workplan

In [9]:
# from workplan_wst, we extract current workplan of all agent on the floor
current_wp = workplan_wst.loc[(pd.to_datetime(workplan_wst['act_starttime'], format='mixed')<=timestamp) & (pd.to_datetime(workplan_wst['act_endtime'], format='mixed')>=timestamp)]

In [10]:
# Create a view where we can check if agent are following the workplan
view_wp = pd.merge(current_wp, rt_view[['Emp ID', 'Reason', 'Duration', 'State', 'time_stamp']], left_on = 'EID', right_on = 'Emp ID', how='left')
view_wp['actual_starttime'] = view_wp.apply(lambda x: '' if pd.isnull(x['Duration']) else timestamp - timedelta(days = x['Duration']), axis=1)
# rearrange the cols
view_wp = view_wp[['LOB', 'Agent ID', 'EID', 'Ted Name', 'Full name', 'Date', 'date_starttime', 'date_endtime', 'Schedule Act','act_starttime', 'act_endtime', 'date_duration', 'Length', 'time_stamp', 'Emp ID', 'Reason', 'Duration', 'State', 'actual_starttime']]
# The remarks
# view_wp['login_status'] = view_wp.apply(lambda x: "not logging as the workplan" if pd.isnull(x['Emp ID']) else 'login', axis=1)

KeyError: "['Ted Name', 'Full name', 'Emp ID'] not in index"

In [ ]:
timestamp = timestamp.strftime("%Y%m%d-%H%M%S")
view_wp.to_csv(os.path.join(r'C:', user_credential, r'Data\realtime_capture', f'capture_{timestamp}.csv'), index=False)